In [1]:
from datetime import datetime, timedelta
import pytz
import math
import pylab as plt

import pandas as pd
import numpy as np

import sys
sys.path.append('../data/')
sys.path.append('../view/')
sys.path.append('../analysis/')

%load_ext autoreload

from extractor import WeatherExtractor, Extractor, Period
sampling_interval = '15min'
import tqdm

%matplotlib widget
from plotter import Plot

from inversegreyboxmodel import Learner

import openpyxl


# TODO: get list of pseudonyms, (valid) dates and reference parameters from Excel file  
# homes =  [817341, 886307, 873985]

homes = [815925, 817341, 857477, 873985, 879481, 
         895671, 864296, 845997, 881611, 822479, 
         831062, 830088, 811308, 809743, 829947, 
         803422, 886307, 846697, 845966, 839440, 
         805164, 899510, 897349]


#location: center of Assendorp neighbourhood in Zwolle
lat, lon = 52.5065500000, 6.0996100000

#timezone: 
timezone_homes = 'Europe/Amsterdam'

# first test just a short timeseries
start = pytz.timezone(timezone_homes).localize(datetime(2021, 12, 20))
end = pytz.timezone(timezone_homes).localize(datetime(2022, 3, 15))

# date and times of seemingly valid periods for short set of 3 
# 886307	2021-12-19 18:55	2022-03-17 09:37
# 873985	2021-12-20 12:54	2022-03-17 17:54
# 817341	2022-01-14 20:27	2022-03-17 18:15

    
# utimately: #min, max dates of the analysis
# start = pytz.timezone(timezone_homes).localize(datetime(2021, 10, 21))
# end = pytz.timezone(timezone_homes).localize(datetime(2022, 5, 12)
# homes = [815925, 817341, 857477, 873985, 879481, 
#          895671, 864296, 845997, 881611, 822479, 
#          831062, 830088, 811308, 809743, 829947, 
#          803422, 886307, 846697, 845966, 839440, 
#          805164, 899510, 897349]

In [2]:
#check whether the datetime object is properly timezone-aware
start

datetime.datetime(2021, 12, 20, 0, 0, tzinfo=<DstTzInfo 'Europe/Amsterdam' CET+1:00:00 STD>)

In [3]:
#check whether the datetime object is properly timezone-aware
end

datetime.datetime(2022, 3, 15, 0, 0, tzinfo=<DstTzInfo 'Europe/Amsterdam' CET+1:00:00 STD>)

In [4]:
%%time 
%autoreload 2

# get geospatially interpolated weather from KNMI
# for Twomes, the Weather for all all homes studies can be approached by a single location
# get the dataframe only once for all homes to save time

df_weather = WeatherExtractor.get_interpolated_weather_nl(start, end, lat, lon, timezone_homes, sampling_interval)

Resampling weather data...
CPU times: user 48.7 ms, sys: 9.48 ms, total: 58.2 ms
Wall time: 56 ms


In [ ]:
df_weather

In [ ]:
# plot temperature data

Plot.weather_and_other_temperatures('Weather in Assendorp, Zwolle', df_weather)

# N.B. The resulting figure below can be manipulated interactively; hover with mouse for tips & tricks

In [5]:
# get interpolated data from the Twomes database and combine with weather data already obtained
df_data_homes = Extractor.get_interpolated_twomes_data(homes, start, end, timezone_homes, sampling_interval, df_weather)

Retrieving data for pseudonyms...


100% 23/23 [21:54<00:00, 57.13s/it]


In [7]:
for col in df_data_homes.select_dtypes(['datetimetz']).columns:
    df_data_homes[col] = df_data_homes[col].dt.tz_localize(None)
df_data_homes.tz_localize(None, level=0).to_excel('df_data_homes_01.xlsx')

In [6]:
# see what the extracted and interpolated data looks like
df_data_homes

,homepseudonym,heartbeat,outdoor_temp_degC,windspeed_m_per_s,effective_outdoor_temp_degC,hor_irradiation_J_per_h_per_cm^2,hor_irradiation_W_per_m^2,indoor_temp_degC,indoor_temp_degC_CO2,indoor_setpoint_temp_degC,gas_m^3,e_used_normal_kWh,e_used_low_kWh,e_returned_normal_kWh,e_returned_low_kWh,timedelta,timedelta_s,e_used_net_kWh,e_remaining_heat_kWh,daycompleteness
2021-12-20 00:00:00+01:00,815925,NaN,6.021051,1.604638,4.951292,-1.976263e-323,-5.434722e-323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:15:00,900.0,NaN,NaN,NaN
2021-12-20 00:15:00+01:00,815925,NaN,5.981809,1.604638,4.912050,-1.976263e-323,-5.434722e-323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:15:00,900.0,NaN,NaN,NaN
2021-12-20 00:30:00+01:00,815925,NaN,5.942567,1.604638,4.872808,-1.976263e-323,-5.434722e-323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:15:00,900.0,NaN,NaN,NaN
2021-12-20 00:45:00+01:00,815925,NaN,5.903324,1.604638,4.833566,-1.976263e-323,-5.434722e-323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:15:00,900.0,NaN,NaN,NaN
2021-12-20 01:00:00+01:00,815925,3.0,5.868830,1.583000,4.813496,-1.976263e-323,-5.434722e-323,NaN,NaN,NaN,0.0,0.0,0.002,0.0,0.0,0 days 00:15:00,900.0,0.002,0.002,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-14 23:45:00+01:00,897349,3.0,2.603058,0.525261,2.252884,-1.976263e-323,-5.434722e-323,NaN,NaN,NaN,0.0,0.0,0.001,0.0,0.0,0 days 00:15:00,900.0,0.001,0.001,1.0
2022-03-15 00:00:00+01:00,897349,3.0,2.644336,0.430313,2.357461,-1.976263e-323,-5.434722e-323,NaN,NaN,NaN,0.0,0.0,0.001,0.0,0.0,0 days 00:15:00,900.0,0.001,0.001,1.0
2022-03-15 00:15:00+01:00,897349,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.007,0.0,0.0,0 days 00:15:00,900.0,0.007,0.007,1.0
2022-03-15 00:30:00+01:00,897349,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.004,0.0,0.0,0 days 00:15:00,900.0,0.004,0.004,1.0


In [8]:
# how to select data from a single home
df_data_homes[df_data_homes['homepseudonym'] == 803422]

,homepseudonym,heartbeat,outdoor_temp_degC,windspeed_m_per_s,effective_outdoor_temp_degC,hor_irradiation_J_per_h_per_cm^2,hor_irradiation_W_per_m^2,indoor_temp_degC,indoor_temp_degC_CO2,indoor_setpoint_temp_degC,gas_m^3,e_used_normal_kWh,e_used_low_kWh,e_returned_normal_kWh,e_returned_low_kWh,timedelta,timedelta_s,e_used_net_kWh,e_remaining_heat_kWh,daycompleteness
2021-12-20 00:00:00+01:00,803422,NaN,6.021051,1.604638,4.951292,-1.976263e-323,-5.434722e-323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:15:00,900.0,NaN,NaN,NaN
2021-12-20 00:15:00+01:00,803422,NaN,5.981809,1.604638,4.912050,-1.976263e-323,-5.434722e-323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:15:00,900.0,NaN,NaN,NaN
2021-12-20 00:30:00+01:00,803422,NaN,5.942567,1.604638,4.872808,-1.976263e-323,-5.434722e-323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:15:00,900.0,NaN,NaN,NaN
2021-12-20 00:45:00+01:00,803422,NaN,5.903324,1.604638,4.833566,-1.976263e-323,-5.434722e-323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:15:00,900.0,NaN,NaN,NaN
2021-12-20 01:00:00+01:00,803422,NaN,5.868830,1.583000,4.813496,-1.976263e-323,-5.434722e-323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:15:00,900.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-14 23:45:00+01:00,803422,3.0,2.603058,0.525261,2.252884,-1.976263e-323,-5.434722e-323,21.446667,NaN,NaN,0.003,0.0,9.094947e-13,0.0,0.0,0 days 00:15:00,900.0,9.094947e-13,9.094947e-13,1.0
2022-03-15 00:00:00+01:00,803422,3.0,2.644336,0.430313,2.357461,-1.976263e-323,-5.434722e-323,21.220000,NaN,NaN,0.000,0.0,9.094947e-13,0.0,0.0,0 days 00:15:00,900.0,9.094947e-13,9.094947e-13,1.0
2022-03-15 00:15:00+01:00,803422,3.0,NaN,NaN,NaN,NaN,NaN,21.006667,NaN,NaN,0.000,0.0,3.000000e-03,0.0,0.0,0 days 00:15:00,900.0,3.000000e-03,3.000000e-03,1.0
2022-03-15 00:30:00+01:00,803422,3.0,NaN,NaN,NaN,NaN,NaN,20.793333,NaN,NaN,0.000,0.0,1.000000e-03,0.0,0.0,0 days 00:15:00,900.0,1.000000e-03,1.000000e-03,1.0


In [ ]:
# plot temperature data of multiple homes from an array 

for home_id in homes:
    df_data_one_home = df_data_homes[df_data_homes['homepseudonym'] == home_id]
    Plot.weather_and_other_temperatures(home_id, df_data_one_home, ['indoor_temp_degC'])

# N.B. The resulting figure below can be manipulated interactively; hover with mouse for tips & tricks


In [ ]:
#plot a series of weeks for all homes
plt.ioff()

for home_id in homes:
    df_data_one_home = df_data_homes[df_data_homes['homepseudonym'] == home_id]
    for moving_horizon_start in pd.date_range(start=start, end=end, inclusive='left', freq='7D'):
        moving_horizon_end = min(end, moving_horizon_start + timedelta(days=7))
        df_moving_horizon = df_data_one_home[moving_horizon_start:moving_horizon_end]
        Plot.weather_and_other_temperatures(home_id, df_moving_horizon, ['indoor_temp_degC'])

plt.ion()  
    

In [14]:
%%time 
%autoreload 2

# uncomment lines below if you need something other than the defaults
homes_to_analyze = [809743]
start_analysis_period = pytz.timezone(timezone_homes).localize(datetime(2021, 12, 31))
end_analysis_period = pytz.timezone(timezone_homes).localize(datetime(2022, 3, 15))
moving_horizon_duration_d = 7
df_results = Learner.learn_home_parameter_moving_horizon(df_data_homes, moving_horizon_duration_d, homes_to_analyze, start_analysis_period, end_analysis_period, showdetails=True)

# or: uncomment lines below if you want to analyse all homes for all periods with a default moving horizon duration of 7 days
# df_results = Learner.learn_home_parameter_moving_horizon(df_data_homes)

df = df_results
for col in df.select_dtypes(['datetimetz']).columns:
    df[col] = df[col].dt.tz_localize(None)
df.to_excel('result.xlsx')

Homes to analyse:  [809743]
Start of analyses:  2021-12-31 00:00:00+01:00
End of analyses:  2022-03-15 00:00:00+01:00
Moving horizon:  7D


  0% 0/1 [00:00<?, ?it/s]

Home pseudonym:  809743



  0% 0/11 [00:00<?, ?it/s]

Start datetime:  2021-12-31 00:00:00+01:00
End datetime:  2022-01-07 00:00:00+01:00


...

 ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           19
   Intermediates:            5
   Connections  :            0
   Equations    :            6
   Residuals    :            1
 
 Number of state variables:          20162
 Number of total equations: -        20160
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              2
 
 solver            3  not supported
 using default solver: APOPT
 ----------------------------------------------
 Dynamic Estimation with APOPT Solver
 ----------------------------------------------
 
 Iter    Objective  Convergence
    0     Infinity  2.32000E+02
    1  1.15886E+04  1.42793E-02
    2  1.4914


 91% 10/11 [27:45<02:27, 147.03s/it]

 ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           19
   Intermediates:            5
   Connections  :            0
   Equations    :            6
   Residuals    :            1
 
 Number of state variables:          20162
 Number of total equations: -        20160
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              2
 
 solver            3  not supported
 using default solver: APOPT
 ----------------------------------------------
 Dynamic Estimation with APOPT Solver
 ----------------------------------------------
 
 Iter    Objective  Convergence
    0     Infinity  2.32000E+02
    1  1.34155E+04  1.53968E-02
    2  1.3557


100% 11/11 [28:16<00:00, 154.21s/it]
100% 1/1 [28:16<00:00, 1696.27s/it]

 ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :           19
   Intermediates:            5
   Connections  :            0
   Equations    :            6
   Residuals    :            1
 
 Number of state variables:          11522
 Number of total equations: -        11520
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              2
 
 solver            3  not supported
 using default solver: APOPT
 ----------------------------------------------
 Dynamic Estimation with APOPT Solver
 ----------------------------------------------
 
 Iter    Objective  Convergence
    0     Infinity  2.08000E+02
    1  5.93224E+03  1.41405E-02
    2  8.6178

In [15]:
#show the results
df_results

,pseudonym,start_horizon,end_horizon,duration_s,error_K,H_W_per_K,tau_h,eta_hs,A_m^2
0,809743,2021-12-31,2022-01-07,604800.0,0.014503,64.337663,92.651385,0.9,6.0
0,809743,2022-01-07,2022-01-14,604800.0,0.016664,75.906336,88.519817,0.9,6.0
0,809743,2022-01-14,2022-01-21,604800.0,0.013058,69.794110,49.912571,0.9,6.0
0,809743,2022-01-21,2022-01-28,604800.0,0.020612,74.073220,21.909223,0.9,6.0
0,809743,2022-01-28,2022-02-04,604800.0,0.054801,53.164977,1000.000000,0.9,6.0
0,809743,2022-02-04,2022-02-11,604800.0,0.015478,52.745962,49.356361,0.9,6.0
0,809743,2022-02-11,2022-02-18,604800.0,0.017096,60.701911,87.447518,0.9,6.0
0,809743,2022-02-18,2022-02-25,604800.0,0.028031,54.853582,244.555188,0.9,6.0
0,809743,2022-02-25,2022-03-04,604800.0,0.018158,79.634503,88.055020,0.9,6.0
0,809743,2022-03-04,2022-03-11,604800.0,0.022003,82.311109,297.268890,0.9,6.0
